## PIPELINE

### unix req:
  - parallel
  - perl, sed, sort, etc.)

### Req.:
  - FastQC
  - MultiQC
  - Kraken2
  - Braken

### python
  - bipython
  - seaborn
  
### Data requeriments
  - Preprocessed, indexed 16S database (SILVA132)
  - Taxonomy (i.e. NCBI)
  

In [1]:
# setting up the environment
import os
import re
from multiprocessing import Pool
from os.path import join, isfile, splitext
from os import listdir
import pandas as pd
from IPython.core.display import display, HTML

import sys
sys.path.append('/home/ligeti/gitrepos/SE-MDR-Bacteria/bin/Klebsiella')

from MicrobiomeAnalysisLibrary import *

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', -1)

%load_ext autoreload
%autoreload 2

base_path = '/gfs/data/SE-OM/Ersebeszet'
save_path = join(base_path, 'raw_fastq/')

# QC outputs
quality_control_output = join(base_path, 'raw_qc/')
quality_control_summary_output = join(base_path, 'raw_multiqc/')
trimmed_quality_control_output = join(base_path, 'trimmed_qc/')
trimmed_quality_control_summary_output = join(base_path, 'trimmed_multiqc/')


# trimming
trimmed_path = join(base_path, 'trimmed_files/')
trimmomatic_path = '/gfs/progs/Trimmomatic-0.36/trimmomatic-0.36.jar'
trimmomatic_adapter_path = '/gfs/progs/Trimmomatic-0.36/adapters/TruSeq3-PE.fa'
# parameters have been set after the QC
trimmomatic_parameters = 'ILLUMINACLIP:{0}:2:12:5 LEADING:3 TRAILING:3 SLIDINGWINDOW:3:15 MINLEN:36 HEADCROP:12'.format(trimmomatic_adapter_path)

# Kraken parameters:
kraken_output_folder = join(base_path, 'trimmed_kraken')

report_postfix = '.report.out'
output_postfix = '.output.out'
db_path = '/gfs/progs/kraken2/SILVAV132NR99'
threads = 20
other_flags = ['--fastq-input']

# Bracken parameters
bracken_output_folder = join(base_path, 'trimmed_bracken')
bracken_db_length = 300
bracken_report_levels = ['S', 'G', 'P','C','F']
bracken_threshold = 20

# Final output
compressed_results = join(base_path, 'compressed_results')



In [5]:
# Creating output directories

if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(quality_control_output):
    os.makedirs(quality_control_output)
if not os.path.exists(quality_control_summary_output):
    os.makedirs(quality_control_summary_output)    
if not os.path.exists(trimmed_path):
    os.makedirs(trimmed_path)    
if not os.path.exists(trimmed_quality_control_output):
    os.makedirs(trimmed_quality_control_output)
if not os.path.exists(trimmed_quality_control_summary_output):
    os.makedirs(trimmed_quality_control_summary_output)

if not os.path.exists(kraken_output_folder):
    os.makedirs(kraken_output_folder)
    
if not os.path.exists(bracken_output_folder):
    os.makedirs(bracken_output_folder)

if not os.path.exists(compressed_results):
    os.makedirs(compressed_results)

## Running QC with fastqc and multiQC

<br>
<img src="Pictures/Pipeline_small1.jpg", width=800>

In [8]:
run_qc(save_path, quality_control_output, quality_control_summary_output, default_fastq_ext='fastq')
qc_table = get_qc_html_table(quality_control_output)


Running raw fastqc: fastqc --outdir /gfs/data/SE-OM/Ersebeszet/raw_qc/      -t 20      /gfs/data/SE-OM/Ersebeszet/raw_fastq/*fastq     
Running raw multiqc: multiqc -f --outdir /gfs/data/SE-OM/Ersebeszet/raw_multiqc/     /gfs/data/SE-OM/Ersebeszet/raw_qc/*     


In [9]:
# Generating links to the results
multi_qc_results_link = '<h1>QC summary:</h1> <a href="https://localhost:8888/files/{0}/multiqc_report.html"> {1} </a> \
 <p> (For details visit the documentation of multiQC <a href="https://multiqc.info"> https://multiqc.info </a>) </p> <br><br>\
 '.format(quality_control_summary_output,'MultiQC results')
display(HTML(multi_qc_results_link))
HTML(qc_table.to_html(escape=False) + '<br><br><br>')



,ID,Forward Reads,Reverse Reads
0,vasc1_S78_L001,vasc1_S78_L001_R1_001_fastqc.html,vasc1_S78_L001_R2_001_fastqc.html
1,vasc2_S79_L001,vasc2_S79_L001_R1_001_fastqc.html,vasc2_S79_L001_R2_001_fastqc.html
2,vasc3_S80_L001,vasc3_S80_L001_R1_001_fastqc.html,vasc3_S80_L001_R2_001_fastqc.html


## Step 2: Preprocessing the reads

<br>
<img src="Pictures/Pipeline_small1.jpg", width=800>


In [10]:
# run trimmomatic
# building the pairs
prefix_to_pair = get_illumina_pairs(save_path)
prefix_to_pair
trim_cmds = [get_trimmomatic_cmd_default(trimmed_path, fastq_files, trimmomatic_path, trimmomatic_parameters) for act_sample, fastq_files in prefix_to_pair.items()]
with  Pool(15) as p:
    p.map(os.system, trim_cmds)
# running QC on filtered data
run_qc(trimmed_path, trimmed_quality_control_output, trimmed_quality_control_summary_output, default_fastq_ext='fastq')
trimmed_qc_table = get_qc_html_table(trimmed_quality_control_output)


Running raw fastqc: fastqc --outdir /gfs/data/SE-OM/Ersebeszet/trimmed_qc/      -t 20      /gfs/data/SE-OM/Ersebeszet/trimmed_files/*fastq     
Running raw multiqc: multiqc -f --outdir /gfs/data/SE-OM/Ersebeszet/trimmed_multiqc/     /gfs/data/SE-OM/Ersebeszet/trimmed_qc/*     


In [11]:
# Generating links to the results
multi_qc_results_link = '<h1>QC summary:</h1> <a href="https://localhost:8888/files/{0}"> {1} </a> \
 <p> (For details visit the documentation of multiQC <a href="https://multiqc.info"> https://multiqc.info </a>) </p> <br><br>\
 '.format(join(trimmed_quality_control_summary_output, 'multiqc_report.html'),'MultiQC results')
display(HTML(multi_qc_results_link))
HTML(trimmed_qc_table.to_html(escape=False) + '<br><br>')

,ID,Forward Reads,Reverse Reads
0,vasc1_S78_L001,vasc1_S78_L001_R1_001.trimmed_fastqc.html,vasc1_S78_L001_R2_001.trimmed_fastqc.html
1,vasc2_S79_L001,vasc2_S79_L001_R1_001.trimmed_fastqc.html,vasc2_S79_L001_R2_001.trimmed_fastqc.html
2,vasc3_S80_L001,vasc3_S80_L001_R1_001.trimmed_fastqc.html,vasc3_S80_L001_R2_001.trimmed_fastqc.html


# Step 4 - Identifying taxa with Kraken2 and Bracken


<br>
<img src="Pictures/Pipeline_small1.jpg", width=800>


In [12]:
prefix_to_trimmed_pair = get_illumina_pairs(trimmed_path)
kraken_report_files = run_kraken2_program(kraken_output_folder, prefix_to_trimmed_pair, 20, db_path, other_flags)

/gfs/progs/kraken2/kraken2                    --report /gfs/data/SE-OM/Ersebeszet/trimmed_kraken/vasc1_S78_L001.report.out                   --output /gfs/data/SE-OM/Ersebeszet/trimmed_kraken/vasc1_S78_L001.output.out                   --db /gfs/progs/kraken2/SILVAV132NR99                   --threads 20                   --fastq-input                   --paired /gfs/data/SE-OM/Ersebeszet/trimmed_files/vasc1_S78_L001_R1_001.trimmed.fastq /gfs/data/SE-OM/Ersebeszet/trimmed_files/vasc1_S78_L001_R2_001.trimmed.fastq
/gfs/progs/kraken2/kraken2                    --report /gfs/data/SE-OM/Ersebeszet/trimmed_kraken/vasc2_S79_L001.report.out                   --output /gfs/data/SE-OM/Ersebeszet/trimmed_kraken/vasc2_S79_L001.output.out                   --db /gfs/progs/kraken2/SILVAV132NR99                   --threads 20                   --fastq-input                   --paired /gfs/data/SE-OM/Ersebeszet/trimmed_files/vasc2_S79_L001_R1_001.trimmed.fastq /gfs/data/SE-OM/Ersebeszet/trimmed_files/

In [13]:
# generating BRACKEN files
report_levels = ['S', 'G', 'P','C','F']
bracken_cmds = []
for act_sample_id, act_report in kraken_report_files.items():
    for act_level in report_levels:
        act_bracken_output = join(bracken_output_folder, act_sample_id + '_' +  act_level + '.bracken_report.txt')
        bracken_cmd = 'bracken \
          -d {0} \
          -i {1} \
          -o {2} \
          -l {3} \
          -r {4} \
          -t {5} \
          '.format(db_path, act_report, act_bracken_output, act_level, bracken_db_length, bracken_threshold)
        
        bracken_cmds.append(bracken_cmd)
        #os.system(bracken_cmd)

with  Pool(threads) as p:
    p.map(os.system, bracken_cmds)

# Step 5 - postprocessing the bracken file

In [3]:
bracken_filenames = sorted([f for path, dfd, filenames in os.walk(bracken_output_folder) for f in filenames if f.endswith('bracken_report.txt')])
# Metadata
metadata_file = join(base_path, 'metadata', 'er_sample_metadata.xlsx')
metadata = pd.read_excel(metadata_file)

# concatenated results
taxa_profile_file = join(base_path, 'taxa_profile.tsv')


In [17]:
metadata.head()

,mouse_id,Dataset prefix,Treatment
0,V1,vasc1_S78,nav
1,V2,vasc2_S79,nav
2,V3,vasc3_S80,nav


In [4]:
# Mapping the results with the metadata

prefixes = list(metadata['Dataset prefix'])
filename2prefixes = {}

for filename in bracken_filenames:
    mapped_ids = []
    filename2prefixes[filename]=[]
    for prefix in prefixes:
        if filename.startswith(prefix): # prefix in filename:
            filename2prefixes[filename].append(prefix)
    if len(filename2prefixes[filename]) > 1:
        print('Unexpected multiplicities!!!!')
        print(filename2prefixes[filename])
    

filename2dataset_id = pd.DataFrame.from_dict(filename2prefixes, orient='index').reset_index()
filename2dataset_id.columns = ['filename', 'dataset_id']

In [5]:
# Reading the files and converting into a common taxa file
bracken_results = []
for filename in bracken_filenames:
    act_bracken = pd.read_csv(join(bracken_output_folder,filename), sep='\t')
    act_bracken['filename'] = filename
    act_bracken_id = act_bracken.merge(filename2dataset_id, how='left', left_on = 'filename', right_on = 'filename')
    bracken_results.append(act_bracken_id)    
bracken_results = pd.concat(bracken_results)
bracken_results = bracken_results.merge(metadata, how='left', left_on = 'dataset_id', right_on = 'Dataset prefix')
bracken_results.to_csv(taxa_profile_file, index=False, sep='\t')

# generate downloadable and portable output


In [7]:
# generate downloadable and portable output
compressed_reports_cmd = 'zip {0} {1}'.format(join(compressed_results, 'kraken2_reports.out.zip'), join(kraken_output_folder, '*.report.out'))
compressed_outputs_cmd = 'zip {0} {1}'.format(join(compressed_results, 'kraken2_output.out.zip'), join(kraken_output_folder, '*.output.out'))
compressed_bracken_results_cmd = 'zip {0} {1}'.format(join(compressed_results, 'bracken_output.out.zip'), join(bracken_output_folder, '*.txt'))


os.system(compressed_reports_cmd)
os.system(compressed_outputs_cmd)
os.system(compressed_bracken_results_cmd)



0